In [ ]:
#importing libraries 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import random 
from random import randint
from itertools import combinations
import matplotlib.pyplot as plt

In [ ]:
##### Read files 
theTotal = []

x = open("REFERENCE-v3.csv")
for line in x:
    line = line.strip()
    a,b = line.split(",")
    theTotal.append(b)
theCsv0 = pd.read_csv("Total_matrix_RR_interval_8528_new_Version.csv", index_col=0)    
#theCsv = pd.read_csv("NewestFeaturesDim01withNopersDattaData.csv",index_col=0)
theCsv = pd.read_csv("NewestFeaturesDim01withNopers50Hz.csv",index_col=0)
#theCsv = pd.read_csv("NewestFeaturesDim01withNopers+NewWin50withnoPers.csv",index_col=0)
theCsv2 = pd.read_csv("RRProxyOldNewPeaks.csv",index_col=0)
theCsv3 = pd.read_csv("RRProxyFromPython.csv",index_col=0)
ratio = pd.read_csv("ratio80Hz.csv",index_col=0)


In [ ]:
#Append Other Features to the main data frame

theCsv["Labels"] = theTotal

#These are the RR interval features
theCsv["SD_RR"] = theCsv0["1"]
theCsv["Mean_RR"] = theCsv0["0"]
theCsv["Skew_RR"] = theCsv0["3"]
theCsv["Kurt_RR"] = theCsv0["2"]

#The ratio of the length of clean data over raw
theCsv["Ratio"] = ratio

#These are features from the Time series that might be good alternatives for the RR features
theCsv["SD_RR_proxy"] = theCsv2["SD_RR_Proxy"]
theCsv["Mean_RR_proxy"] = theCsv2["Mean_RR_Proxy"]
theCsv["Skew_RR_proxy"] = theCsv2["Sk_RR_Proxy"]
theCsv["Kurt_RR_proxy"] = theCsv2["Kurt_RR_Proxy"]
theCsv["Mean_RR_p"] = theCsv3["M_p"]
theCsv["SD_RR_p"] = theCsv3["SD_p"]
theCsv["Skew_RR_p"] = theCsv3["Sk_p"]
theCsv["Kurt_RR_p"] = theCsv3["Kurt_p"]
theCsv["Mean_RR_p2"] = theCsv3["M_p2"]
theCsv["SD_RR_p2"] = theCsv3["SD_p2"]
theCsv["Skew_RR_p2"] = theCsv3["Sk_p2"]
theCsv["Kurt_RR_p2"] = theCsv3["Kurt_p2"]
theCsv["Mean_RR_p3"] = theCsv3["M_p3"]
theCsv["SD_RR_p3"] = theCsv3["SD_p3"]
theCsv["Skew_RR_p3"] = theCsv3["Sk_p3"]
theCsv["Kurt_RR_p3"] = theCsv3["Kurt_p3"]


## Create Test Set with same districution as the hidden test set.

In [ ]:
#Function for selecting ECG for test set

def raffle(n,S,X,Y):
    random.seed(40)
    theTot = np.array(X)
    ind = np.array(np.where(theTot==S)[0])
    ind.tolist()
    numlist = random.sample(range(0, len(ind)-1), n)
    to_remove = ind[numlist]
    Y.append(to_remove)
    return Y 

In [ ]:
#Do the Raffle

Numbers = np.array([78,666,187,69])
AF = raffle(78,"A",theTotal,[])
N = raffle(666,"N",theTotal,[])
O = raffle(187,"O",theTotal,[])
P = raffle(69,"~",theTotal,[])

#Join up and collect labels
TestSetind = np.concatenate((AF,N,O,P), axis=None)
TestSet = np.array([theTotal[k] for k in TestSetind])

#This is just an auxilliary data frame. This is not the final data frame for the test set.
TestSet = pd.DataFrame(TestSet)
TestSetind = pd.DataFrame(TestSetind)
TestSetDF = pd.concat([TestSetind,TestSet], axis=1)
TestSetDF.columns = ['Index',"Class"]

In [ ]:
#Create Test and Training Sets
trainSet = pd.DataFrame([],columns = theCsv.columns)
trainSetInd = []
testSet = pd.DataFrame([],columns = theCsv.columns)
testSetInd = []
for k in range(0,8528):
    if k not in list(TestSetDF["Index"]):
        trainSet = trainSet.append(theCsv.iloc[k],ignore_index=True)
        trainSetInd.append(k)
    else:
        testSet = testSet.append(theCsv.iloc[k],ignore_index=True)
        testSetInd.append(k)


In [ ]:
#Reconfigure Training and Test set for Random Forest and extract Labels out

#append the corresponding original indices for the training and test set.
trainSet["Index"] = trainSetInd
testSet["Index"] = testSetInd

#Exclude zero rows in training
trainSet = trainSet[(trainSet.T != 0).any()]

#Transform row Index using the column "Index"
trainSet.rename(trainSet["Index"], axis = "rows")
testSet.rename(testSet["Index"], axis = "rows")

#Remove "Index column"
trainSet = trainSet.drop("Index",axis=1)
testSet = testSet.drop("Index",axis=1)

#save labels
train_labels = trainSet["Labels"]
test_labels = testSet["Labels"]

#Drop labels in preparation for random forest
trainSet = trainSet.drop("Labels",axis=1)
testSet = testSet.drop("Labels",axis=1)
#trainSet = trainSet.drop("Ratio",axis=1)
testRatio = testSet["Ratio"]


## Begin brute force feature selection

In [ ]:
#Save column names
cols= list(trainSet.columns)

#Remove columns that you want to keep. 
#Yes you read that right, we will re-append this later. We remove them for now so they're not inlcuded in the shuffle of features.

cols.remove('SD_RR')
cols.remove('Skew_RR')
cols.remove('Mean_RR')
cols.remove('Kurt_RR')


In [ ]:
#Initialize 
Scores = []
Feats = []
MaxSoFar=0
rf = RandomForestClassifier(bootstrap=True,max_depth=10,max_features="sqrt",min_samples_leaf=2,min_samples_split=5,n_estimators=600,random_state=40,class_weight="balanced")

#This is the brute force feature selection. It selects a subset of the list of features then trains the forest using the selected.
for d in range(0,len(cols)):
    setfeats = list(combinations(cols,len(cols)-d))
    for s in range(len(setfeats)):
        toappend = list(setfeats[s])
        
        #re-append the features that you want to always be included in the forest
        toappend.append('SD_RR')
        toappend.append('Mean_RR')
        toappend.append('Skew_RR')
        toappend.append('Kurt_RR')
       
        FEAT = trainSet[np.array(toappend)]
        predict_list = []
        rf.fit(FEAT,train_labels)
        predictions = rf.predict(testSet[np.array(toappend)])        
        
        #Override the predictions for ECG whose features are all zeroes as consequence of cleanup.
        predictions2 = np.array(list(predictions))
        predictions2[np.where(testRatio==0)] = '~'
        
        predict_list.append(predictions)
        theMax = 0
        if(accuracy_score(test_labels, predict_list[0]) > theMax):
            
            #The score of the original predictions
            theMax = np.average(f1_score(test_labels, predictions, average=None)[:3])
            
            #The score after overriding some 
            theMax2 = np.average(f1_score(test_labels, predictions2, average=None)[:3])
            
        #If a maximum score is found, print a message 
        if theMax > MaxSoFar:
            MaxSoFar = theMax
            numfeat = len(trainSet.columns)-d
            numfeat = str(numfeat)
            M = MaxSoFar*100
            M = str(M)
            F1A = f1_score(test_labels, predictions, average=None)[0]
            F1N = f1_score(test_labels, predictions, average=None)[1]
            F1O = f1_score(test_labels, predictions, average=None)[2]
            F1P = f1_score(test_labels, predictions, average=None)[3]
            F1A = str(F1A)
            F1N = str(F1N)
            F1O = str(F1O)
            F1P = str(F1P)
            
            MaxSoFar2 = theMax2
            M2 = MaxSoFar2*100-MaxSoFar*100
            M2 = str(M2)
            F1A2 = f1_score(test_labels, predictions2, average=None)[0]-f1_score(test_labels, predictions, average=None)[0]
            F1N2 = f1_score(test_labels, predictions2, average=None)[1]-f1_score(test_labels, predictions, average=None)[1]
            F1O2 = f1_score(test_labels, predictions2, average=None)[2]-f1_score(test_labels, predictions, average=None)[2]
            F1P2 = f1_score(test_labels, predictions2, average=None)[3]-f1_score(test_labels, predictions, average=None)[3]
            F1A2 = str(F1A2)
            F1N2 = str(F1N2)
            F1O2 = str(F1O2)
            F1P2 = str(F1P2)
            
            #second line says "Effect of Noise Isolation". It prints the difference of when there is overring from whene there is none.
            #Positive means the override is good.
            print("Best: "+M +" with " + numfeat + " features and " + "F1A=" + F1A+ "; F1N=" + F1N+ "; F1O="+F1O+ "; F1P=" + F1P +
                 "\n Ef. '~' Iso: "+M2 +"; F1A=" + F1A2+ "; F1N=" + F1N2+ "; F1O="+F1O2+ "; F1P=" + F1P2)
        #Save Scores and Features
        Scores.append(np.concatenate(([len(trainSet.columns)-d,theMax],f1_score(test_labels, predictions, average=None)[0:4]),axis=None))
        Feats.append(np.array(toappend))
        
        FeatsDF = pd.DataFrame(Feats)    
        ScoresDF = pd.DataFrame(Scores)
    
        #Save files. Make sure you change the file name for different Feature Spaces
        ScoresDF.to_csv("ScoresDim01NoPers50HzwithRRFeatures.csv")
        FeatsDF.to_csv("FeatsDim01NoPers50HzwithRRFeatures.csv")